In [8]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from pprint import pprint
from datetime import datetime
import re
from bson.objectid import ObjectId
import os

executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)

In [2]:
def dateFixer(str):
    if str == '':
        return ['', '', '', '']
    else:
        date_list = []
        str = re.sub(r'[.]', ':', str)
        parts = str.split()
        parts[2] = parts[2].strip("stndrh")
        str = " ".join(parts)
        date = re.search(r'^(.*?)\d\d\d\d', str).group(0)
        start_time = re.search(r'.+?(?=–)', str).group(0)
        end_time = date + " " + re.search(r'(?<=–).*', str).group(0)
        start_dt_obj = datetime.strptime(start_time, '%a %b %d %Y %I:%M%p')
        end_dt_obj = datetime.strptime(end_time, '%a %b %d %Y %I:%M%p')
        date_list.append(start_time)
        date_list.append(start_dt_obj)
        date_list.append(end_time)
        date_list.append(end_dt_obj)
        date_list.append(date)
        return date_list


executable_path = {'executable_path': '/Users/soria/Anaconda3/Scripts/chromedriver'}
browser = Browser('chrome', **executable_path)

# Get genres of each program and write to dictionary, match character cases
cacheFilePath = "./../../music_project_caches/_schedule_genre.txt"
if os.path.isfile(cacheFilePath):
    with open(cacheFilePath, encoding='utf-8') as cacheFile:
        html = cacheFile.read()
else:
    browser.visit('http://kdhx.org/shows/schedule')
    browser.is_element_present_by_css("p.plhead", wait_time=1)
    html = browser.html
    with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
        cacheFile.write(html)

schedule_soup = bs(html, 'html.parser')

weekdays = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
kdhx_genres = {}
for day in weekdays:
    schedule = schedule_soup.find('div', class_='day', dow=day).find_all('a')
    program_list = []
    for item in schedule:
        program = item.find('div', class_='show-title').find('span', style='margin-right:5px').get_text().strip()
        program_list.append(program)
        #print(program)
        try:
            program = re.search(r'.+?(?=\s\s)', program).group(0)
        except:
            program = program
        kdhx_genres.setdefault(program, {})
        genre_elements = item.find('div', class_='show-genres').find_all('span')
        counter = 0
        for element in genre_elements:
            counter += 1
            countStr = str(counter)
            kdhx_genres[program].setdefault(countStr, {})
            kdhx_genres[program][countStr] = element.get_text()

kdhx_genres['No Time To Tarry Here'] = kdhx_genres.pop('No Time to Tarry Here')
kdhx_genres['Music from the Hills'] = kdhx_genres.pop('Music From the Hills')
kdhx_genres['Boogie On Down'] = kdhx_genres.pop('Boogie on Down')
kdhx_genres['Howzit Bayou'] = kdhx_genres.pop('Howzit Bayou?')
kdhx_genres['Cure For Pain'] = kdhx_genres.pop('Cure for Pain')
kdhx_genres["Shake 'em on Down"] = kdhx_genres.pop("Shake 'Em on Down")

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.kdhx
collection = db.genres
collection.insert_one(kdhx_genres)
# code to get playlists

cacheFilePath = "./../../music_project_caches/_schedule_playlist.txt"
if os.path.isfile(cacheFilePath):
    with open(cacheFilePath, encoding='utf-8') as cacheFile:
        html = cacheFile.read()
else:
    url = "https://spinitron.com/radio/playlist.php?station=kdhx&show=schedule&sv=l#here"
    browser.is_element_present_by_css("p.plhead", wait_time=1)
    browser.visit(url)
    with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
        cacheFile.write(html)

html = browser.html
schedule_soup = bs(html, 'html.parser')

schedule_tables = schedule_soup.find_all('tbody')
schedule_tables = schedule_tables[2:9]

program_url_list = []
for day in schedule_tables:
    day_list = []
    day_list = day.find_all('p', class_='showname')
    for program in day_list:
        program_url_list.append('https://spinitron.com/radio/' + program.find('a')['href'])

kdhx_programs = {}
test_programs = {}

program_names = [x for x in kdhx_genres.keys()]
for index, program_url in enumerate(program_url_list):
#     print(program_url)
    name = re.sub( r'[\s]', '_', program_names[index])
    name = re.sub( r'[?]', '', name)
    cacheFilePath = f"./../../music_project_caches/_playlists_{name}.txt"
    if os.path.isfile(cacheFilePath):
        with open(cacheFilePath, encoding='utf-8') as cacheFile:
            html = cacheFile.read()
    else:
        browser.visit(program_url)
        browser.is_element_present_by_css("p.plhead", wait_time=1)
        html = browser.html
        with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
            cacheFile.write(html)
    program_soup = bs(html, 'html.parser')
    program_name = program_soup.find('p', class_='plhead').get_text()
    kdhx_programs.setdefault(program_name, {})
    kdhx_programs[program_name].setdefault('program_url', program_url)
    kdhx_programs[program_name].setdefault('genre(s)', {})
    kdhx_programs[program_name].setdefault('playlist(s)', {})
    test_programs.setdefault(program_name, {})
    test_programs[program_name].setdefault('program_url', program_url)
    test_programs[program_name].setdefault('genre(s)', {})
    test_programs[program_name].setdefault('playlist(s)', {})
    playlist_table = program_soup.find('table', id='pltable')
    playlist_anchors = playlist_table.find_all('a', title='Click for the playlist')
    playlist_url_list = []
    counter = 0
    for index, anchor in enumerate(playlist_anchors):
        if index % 2 == 0:
            counter += 1
            countStr = str(counter)
            url = 'https://spinitron.com/radio/' + playlist_anchors[index]['href']
            kdhx_programs[program_name]['playlist(s)'].setdefault(countStr, {})
            kdhx_programs[program_name]['playlist(s)'][countStr] = url
    for index, anchor in enumerate(playlist_anchors[::80]):
        if index % 2 == 0:
            counter += 1
            countStr = str(counter)
            url = 'https://spinitron.com/radio/' + playlist_anchors[index]['href']
            test_programs[program_name]['playlist(s)'].setdefault(countStr, {})
            test_programs[program_name]['playlist(s)'][countStr] = url

program_name_list = [k for k, v in kdhx_programs.items()]
print(test_programs)
print(kdhx_genres)
for program in program_name_list:
    test_programs[program]['genre(s)'] = kdhx_genres[program]

for program in program_name_list:
    kdhx_programs[program]['genre(s)'] = kdhx_genres[program]

{}
{'Elevated Rhymestate': {'1': 'Hip Hop'}, 'Latin Hemispheres': {'1': 'World'}, 'After Hours': {'1': 'Electronic'}, 'Sunday Morning Show': {'1': ''}, 'Lotsa A Cappella': {'1': 'Folk', '2': 'World'}, 'Folks of the World': {'1': 'Folk', '2': 'World'}, 'Songwriters Showcase': {'1': 'Folk', '2': 'Singer/Songwriter'}, 'Bluegrass Breakdown': {'1': 'Bluegrass', '2': 'Folk'}, 'Chicken Shack': {'1': 'Blues', '2': 'R&B', '3': 'Soul'}, "Nothin' but the Blues": {'1': 'Blues'}, 'All Soul, No Borders': {'1': 'Jazz', '2': 'Soul'}, 'In the Middle of the Night': {'1': 'Jazz'}, "The Lion's Den": {'1': 'Rock'}, 'Sonic Space': {'1': 'Funk', '2': 'Hip Hop', '3': 'R&B', '4': 'Rock', '5': 'Electronic'}, 'Mystery Train': {'1': 'Country', '2': 'Folk', '3': 'Rock'}, 'Radio Rio': {'1': 'World'}, 'Dangerous Curves': {'1': 'Blues', '2': 'R&B', '3': 'Rock', '4': 'Soul'}, 'Soul Selector': {'1': 'Blues', '2': 'Soul'}, 'Music at Work': {'1': 'Americana', '2': 'Rock'}, 'loudQUIETloud': {'1': 'Rock'}, 'Serious Journal

In [28]:
cacheFilePath = "./../../music_project_caches/_schedule_playlist.txt"
if os.path.isfile(cacheFilePath):
    with open(cacheFilePath, encoding='utf-8') as cacheFile:
        html = cacheFile.read()
else:
    url = "https://spinitron.com/radio/playlist.php?station=kdhx&show=schedule&sv=l#here"
    browser.is_element_present_by_css("p.plhead", wait_time=1)
    browser.visit(url)
    html = browser.html
    with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
        cacheFile.write(html)


schedule_soup = bs(html, 'html.parser')

schedule_tables = schedule_soup.find_all('tbody')
schedule_tables = schedule_tables[2:9]

program_url_list = []
for day in schedule_tables:
    day_list = []
    day_list = day.find_all('p', class_='showname')
    for program in day_list:
        program_url_list.append('https://spinitron.com/radio/' + program.find('a')['href'])

print(program_url_list)

[]


In [27]:
schedule_soup.find_all('tbody')

[<tbody><tr>
 <td class="eb_event">
 <div class="event-title">
 <a class="eb_event_link" href="/events/kdhx-recommends/wednesday-jazz-crawl-at-kdhx-with-jacob-alspach-friends">
 							Wednesday Jazz Crawl at KDHX with Jacob Alspach &amp; Friends						</a>
 </div>
 <span class="event_date">
 							01-30-2019 5:00 pm						</span>
 <br/>
 <a class="eb-colorbox-map cboxElement" href="/events/the-bar-at-kdhx/view-map?tmpl=component&amp;format=html">The Bar at KDHX</a>
 </td>
 </tr>
 <tr>
 <td class="eb_event">
 <div class="event-title">
 <a class="eb_event_link" href="/events/kdhx-recommends/mvstermind-live-from-the-listening-room">
 							Mvstermind: Live from the Listening Room						</a>
 </div>
 <span class="event_date">
 							01-31-2019 8:00 pm						</span>
 <br/>
 <a class="eb-colorbox-map cboxElement" href="/events/the-stage-at-kdhx/view-map?tmpl=component&amp;format=html">The Stage at KDHX</a>
 </td>
 </tr>
 <tr>
 <td class="eb_event">
 <div class="event-title">
 <a class="eb_ev

In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.kdhx
collection = db.urls_test
test_programs = collection.find_one()
del test_programs['_id']
test_programs

{'Sunday Morning Show': {'program_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&showid=456',
  'genre(s)': {'1': ''},
  'playlist(s)': {'2': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=60537#here'}},
 'Lotsa A Cappella': {'program_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&showid=15',
  'genre(s)': {'1': 'Folk', '2': 'World'},
  'playlist(s)': {'446': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62739#here',
   '447': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62553#here',
   '448': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62367#here',
   '449': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62184#here',
   '450': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62017#here',
   '451': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=61840#here'}},
 'Folks of the World': {'program_u

In [4]:
program_name_list = [k for k, v in test_programs.items()]
db = client.kdhx
collection = db.test
kdhx_dict = {}
for program in program_name_list[0:5]:
    for index, playlist_url in test_programs[program]['playlist(s)'].items():
        file = re.sub(r'[\s]', '_', program)
        cacheFilePath = f"./../../music_project_caches/{file}_{index}.txt"
        if os.path.isfile(cacheFilePath):
            with open(cacheFilePath, encoding='utf-8') as cacheFile:
                html = cacheFile.read()
        else:
            browser.visit(playlist_url)
            browser.is_element_present_by_css("p.plhead", wait_time=1)
            html = browser.html
            with open(cacheFilePath, "w", encoding='utf-8') as cacheFile:
                cacheFile.write(html)
        print(f"{program} {index} of {len(test_programs[program]['playlist(s)'])}: {playlist_url}")
        playlist_soup = bs(html, 'html.parser')
        try:
            if playlist_soup.find('p', class_='plhead').get_text() == ('Chicken Shack' or 'Chicken Shack Alley'):
                program_name = 'Chicken Shack'
            else:
                program_name = playlist_soup.find('p', class_='plhead').get_text()
        except:
            program_name = ''
        try:
            play_date = playlist_soup.find('p', class_='plheadsub').get_text()
        except:
            play_date = ''
        try:
            dj = playlist_soup.find('div', class_='infoblock').find('p', class_='plhead').find('a').get_text()
            dj = re.sub(r'[.]', '', dj)
        except:
            dj = ''
        start_time = dateFixer(play_date)[0]
        start_dto = dateFixer(play_date)[1]
        end_time = dateFixer(play_date)[2]
        end_dto = dateFixer(play_date)[3]
        date = dateFixer(play_date)[4]
        kdhx_dict.setdefault(program_name, {}).setdefault(start_time, {}).setdefault(dj, {})
        kdhx_dict[program_name]['start_dt_object'] = start_dto
        kdhx_dict[program_name]['end time'] = end_time
        kdhx_dict[program_name]['end_dt_object'] = end_dto
        kdhx_dict[program_name]['genre(s)'] = test_programs[program]['genre(s)']
        try:
            kdhx_dict[program_name]['description'] = playlist_soup.find('div', id='playlisthead').find('p', class_='indent').get_text()
        except:
            kdhx_dict[program_name]['description'] = ""
        play_dict = {}
        playlist_div = playlist_soup.find('div', class_='plblock')
        play_divs = playlist_div.find_all('div', class_='f2row')
        for index, play in enumerate(play_divs):
            try:
                play_time = play.find('p', class_='st').get_text()
                play_time_obj = datetime.strptime(date + " " + play_time, '%a %b %d %Y %I:%M%p')
            except:
                play_time = '' 
                play_time_obj = ''
            try:
                if index == len(play_divs) - 1:
                    time_delta = datetime.strptime(re.search(r'(?<=\d\d\d\d\s).*', end_time).group(0), '%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
                else:
                    time_delta = datetime.strptime(play_divs[index+1].find('p', class_='st').get_text(),'%I:%M%p') - datetime.strptime(play_time,'%I:%M%p')
                play_duration = time_delta.total_seconds() / 60
            except:
                play_duration = ''
            kdhx_dict[program_name][start_time][dj].setdefault(play_time, {})
            play_dict = {'artist': '', 'artist_url': '', 'track': '', 'album': '', 'album_url': '',    
                         'label': '', 'label_url': '', 'type': '', 'notes': '', 'play_duration': '', 'play_time_obj': ''}
            play_dict['play_time_obj'] = play_time_obj
            play_dict['play_duration'] = play_duration
            try:
                play_dict['artist'] = play.find('span', class_='aw').get_text()
            except:
                play_dict['artist'] = ''
            try:
                play_dict['artist_url'] = "https://spinitron.com/radio/" + play.find('span', class_='aw').find('a')['href']
            except:
                play_dict['artist_url'] = ''
            try:
                play_dict['track'] = play.find('span', class_='sn').get_text()
            except:
                play_dict['track'] = ''
            try:
                play_dict['album'] = play.find('span', class_='dn').get_text()
            except:
                play_dict['album'] = ''
            try:
                play_dict['album_url'] = "https://spinitron.com/radio/" + play.find('span', class_='dn').find('a')['href']
            except:
                play_dict['album_url'] = ''
            try:
                play_dict['label'] = play.find('span', class_='ld').get_text()
            except:
                play_dict['label'] = ''
            try:
                play_dict['label_url'] = "https://spinitron.com/radio/" + play.find('span', class_='ld').find('a')['href']
            except:
                play_dict['label_url'] = ''
            try:
                play_dict['type'] = play.find('span', class_='fg').get_text()
            except:
                play_dict['type'] = ''
            try:
                play_dict['notes'] = play.find('span', class_='so').get_text()
            except:
                play_dict['notes'] = ''
            kdhx_dict[program_name][start_time][dj][play_time] = play_dict
        collection.update_one({}, {'$set': kdhx_dict}, upsert=True)

Sunday Morning Show 2 of 1: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=60537#here
Lotsa A Cappella 446 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62739#here
Lotsa A Cappella 447 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62553#here
Lotsa A Cappella 448 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62367#here
Lotsa A Cappella 449 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62184#here
Lotsa A Cappella 450 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62017#here
Lotsa A Cappella 451 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=61840#here
Folks of the World 425 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62742#here
Folks of the World 426 of 6: https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&playlist=62554#here
Folks of the World 427 of 6: https://spin

In [6]:
from pprint import pprint
pprint(kdhx_dict)

{'Bluegrass Breakdown (Music)': {'Sun Dec 23 2018 12:00pm': {'Walter and Willa': {'12:00pm': {'album': 'Christmas '
                                                                                                       'with '
                                                                                                       'Echo '
                                                                                                       'Valley '
                                                                                                       '- '
                                                                                                       'EP',
                                                                                              'album_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=d&dbid=441979#here',
                                                                                              'artist': 'Echo '
                       

                                                                                              'artist': 'High '
                                                                                                        'Fidelity',
                                                                                              'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=100998#here',
                                                                                              'label': '(High '
                                                                                                       'Fidelity '
                                                                                                       '2018)',
                                                                                              'label_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=l&dbid=7073#here',
                                        

                                                                                                       'Of '
                                                                                                       'Jo',
                                                                                              'album_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=d&dbid=146944#here',
                                                                                              'artist': 'Greg '
                                                                                                        'Cahill',
                                                                                              'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=59247#here',
                                                                                              'label': '(Hueber '
                                                

                                                                                              'play_duration': 3.0,
                                                                                              'play_time_obj': datetime.datetime(2019, 1, 20, 12, 8),
                                                                                              'track': '“Ring '
                                                                                                       'of '
                                                                                                       'Fire”',
                                                                                              'type': ''},
                                                                                  '12:11pm': {'album': 'Tomorrow',
                                                                                              'album_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=d&d

                                                                                              'track': '“Poor '
                                                                                                       'Little '
                                                                                                       'Sadie”',
                                                                                              'type': ''},
                                                                                  '12:21pm': {'album': 'Sasquatch '
                                                                                                       'with '
                                                                                                       'an '
                                                                                                       'Alien '
                                                                                                       'on 

...

                                                                                     '10:20am': {'album': 'Wings',
                                                                                                 'album_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=d&dbid=416932#here',
                                                                                                 'artist': 'Zoe '
                                                                                                           'Speaks',
                                                                                                 'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=20301#here',
                                                                                                 'label': '(Redbird '
                                                                                                          'Records '
                   

                                                                                                  'Cold '
                                                                                                  'Hearts',
                                                                                        'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=146412#here',
                                                                                        'label': '(Lost '
                                                                                                 'Highway '
                                                                                                 'Australia '
                                                                                                 '2018)',
                                                                                        'label_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbd

                                                                                        'play_time_obj': datetime.datetime(2019, 1, 20, 10, 41),
                                                                                        'track': '“the '
                                                                                                 'Tide”',
                                                                                        'type': ''},
                                                                            '10:46am': {'album': 'Eagle '
                                                                                                 'In '
                                                                                                 'the '
                                                                                                 'Rain',
                                                                                        'album_url': 'https://spinitron.com/r

                                                                                                '2017)',
                                                                                       'label_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=l&dbid=69034#here',
                                                                                       'notes': '',
                                                                                       'play_duration': 6.0,
                                                                                       'play_time_obj': datetime.datetime(2019, 1, 6, 10, 20),
                                                                                       'track': '“Nothing '
                                                                                                'to '
                                                                                                'Escape”',
                                     

                                                                                                     '& '
                                                                                                     'Champion '
                                                                                                     'Jack '
                                                                                                     'Dupree',
                                                                                           'artist_url': 'https://spinitron.com/radio/playlist.php?station=kdhx&sv=l&dbdetail=a&dbid=152052#here',
                                                                                           'label': '(Rock '
                                                                                                    'Beat '
                                                                                                    'Records '
                                      